#### Домашнее задание к занятию "Коллаборативная фильтрация"

ПАКЕТ SURPRISE  
− используйте данные MovieLens 1M  
− можно использовать любые модели из пакета  
− получите RMSE на тестовом сете 0.87 и ниже

\* В домашнем задании использованы данные обычного MovieLens, поскольку скачивание 1М из-за большого объема (через раздачу мобильного интернета) постоянно срывалось где-то на середине. А задание нужно сдавать. Когда будет возможность, обязательно подставлю данные большого датасета.

In [1]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [2]:
movies = pd.read_csv('C:/Users/gogol/Desktop/Netology/I work in this folder/Блок 4 Машинное обучение/12 Введение и классификация рекомендательных систем/movies.csv')
ratings = pd.read_csv('C:/Users/gogol/Desktop/Netology/I work in this folder/Блок 4 Машинное обучение/12 Введение и классификация рекомендательных систем/ratings.csv')

Объединим датасеты и избавимся от пропущенных значений

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

И немного его видоизменим

In [4]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})
dataset.head()

,uid,iid,rating
0,7.0,Toy Story (1995),3.0
1,9.0,Toy Story (1995),4.0
2,13.0,Toy Story (1995),5.0
3,15.0,Toy Story (1995),2.0
4,19.0,Toy Story (1995),3.0


Посмотрим на минимальные и максимальные оценки фильмов

In [5]:
print('min rating =', ratings.rating.min())
print('max rating =', ratings.rating.max())

min rating = 0.5
max rating = 5.0


Воспользуемся библиотекой surprise

In [6]:
# The Reader class is used to parse a file containing ratings. Default rating_scale is (1, 5)
reader = Reader(rating_scale=(0.5, 5.0))

# To load a dataset from a pandas dataframe, you will need the load_from_df() method.
data = Dataset.load_from_df(dataset, reader)

# If you don’t want to run a full cross-validation procedure, you can use the train_test_split() to sample a trainset and a testset with given sizes
trainset, testset = train_test_split(data, test_size=.15)

Воспользуемся user-based collaborative filtering и алгоритмом прогнозирования knns.KNNBaseline, о котором полную информацию дает сайт https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html#module-surprise.prediction_algorithms

In [7]:
algo = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

Обучим модель на тренировочных данных

In [8]:
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


И на тестовых данных получим прогнозы

In [9]:
test_pred = algo.test(testset)

Рассчитаем точность прогноза с помощью среднеквадратичной ошибки RMSE

In [10]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9053


0.905342644883184